In [167]:
import re
import string
from cleantext import clean
import requests
import json
import glob
import os
import pandas as pd
import numpy as np
import dateparser
import string 
#Data cleaning 
import bs4 as bs
import urllib.request
import re
import nltk

In [168]:
#!pip install clean-text
#!pip install unidecode

In [169]:
def clean(file):
    article_text = re.sub(r'\[[0-9]*\]', ' ', file)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    return  article_text,formatted_article_text


In [170]:
text_path = "./7100-15/cftc_text_7100-15_press-release.txt"
def load_data(text_path):
    with open(text_path) as f:
        file = f.read()
    return file
file=load_data(text_path)
doc,format_=clean(file)

In [171]:
def unicode(text):
    text_unicode = doc 
    # encoding the text to ASCII format
    text_encode = text_unicode.encode(encoding="ascii", errors="ignore")
    # decoding the text
    text_decode = text_encode.decode()
    # cleaning the text to remove extra whitespace 
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

In [172]:
text=unicode(doc)

In [173]:
sentence_list = nltk.sent_tokenize(text)


In [174]:
cleaned_text = ' '.join(sentence_list)


In [175]:
cleaned_text

'Release Number 7100-15 January 12, 2015 Federal Court in Florida Enters Order Freezing Assets in CFTC Foreign Currency Anti- Fraud Action against Allied Markets LLC and its Principals Joshua Gilliland and Chawalit Wongkhiao CFTC Charges Jacksonville, Florida, Defendants with Operating a Fraudulent Forex Pool and Misappropriating Customer Funds to Pay Personal Expenses Washington, DC The U.S. Commodity Futures Trading Commission (CFTC) today announced that it filed a civil enforcement Complaint in the U.S. District Court for the Middle District of Florida, charging Defendants Allied Markets LLC, and its principals Joshua Gilliland and Chawalit Wongkhiao, all of Jacksonville, Florida, with operating a fraudulent foreign currency (forex) commodity pool in violation of the Commodity Exchange Act (CEA) and CFTC Regulations. In addition, none of the Defendants has ever been registered with the CFTC, as required. On January 7, 2015, the day after the CFTC Complaint was filed under seal, U.S.

In [176]:
def is_section_num(text):
    
    return re.search(r'^[MDCLXVIAB]+\.$',text) is not None


In [177]:
 is_section_num(doc)

False

In [178]:
sentence_scores=sentence_score(cleaned_text)

TypeError: sentence_score() missing 1 required positional argument: 'word_frequencies'

In [179]:
def ends_with_ponc(text):
    return text[-1] in punctuation if text else False 
#ends_with_ponc("Propex engaged in sanctions.")

In [180]:
def start_with_upper(text):
    return re.match('^[A-Z]', text) is not None
start_with_upper("Fsfsd.")

True

In [181]:
def is_title(text):
    uppers = [word.isupper() for word in text.split(" ")]
    return sum(uppers) / len(uppers) > 0.6
is_title("UNITED STATES OF AMERICA Before the COMMODITY FUTURES TRADING COMMISSION")

True

In [182]:
import nltk
from nltk.tokenize import sent_tokenize
t = sent_tokenize("Three days later, the SEC  filed a federal injunctive action against Sentinel, SEC v.  Sentinel,  et al., No.  07 CV 4684 (N.D. Ill. filed Aug. 20, 2007), and on April 28, 2008, the CFTC filed a Complaint against Sentinel, Bloom and Sentinel's Senior Vice President and head trader, Charles K. Mosley, seeking injunctive and other equitable relief, as well as the imposition of civil penalties, for violating various provisions of the Act and Commission Regulations.")

In [98]:
def same_sentence(sent1, sent2):
    #print(sent1,"\n",sent2)
    
    # sent empty
    if (not sent1) or (not sent1['text']) :
        return True
    # section number
    if sent1['is_section_num']:
        return True
    
    # very short sentence
    if len(sent1['text']) < 50:
        return False
    
    # ponctuation
    if sent1['ends_with_ponc']:
        return False
    
    if sent1['is_title']:
        return False

    if sent2['is_title']:
        return False
    
    return True

In [ ]:
punctuation=['.','?','!']

In [147]:
def clean_doc(file_text):
    pages=[]
    for page in file_text.split("\x0c"):
        # clen text
        page_meta = [{'text': clean(para)} 
                            for para in page.split('\n')]
        clean_page = []
        previous_line = {}
        text= ""
        # add meta data
        for line in page_meta:
            line['is_section_num'] = is_section_num(str(line['text']))
            line['is_title'] = is_title(str(line['text']))
            line['ends_with_ponc'] = ends_with_ponc(str(line['text']))
            line['is_alpha'] = sum(c.isalpha() for c in str(line['text']))
            line['start_with_upper']  = start_with_upper(str(line['text']))
                                
            # not relevant line
            if not line['is_alpha']:
                continue
            
            if not same_sentence(previous_line, line):
                if text:
                    clean_page.append(text)
                    text = ""
                
            previous_line = line
            text = " ".join([text, str(line['text'])])
        if len(clean_page):
            pages.append(clean_page)
    return pages

In [250]:
def get_structured_document(file):
    text_structured=[]
    file_cleaned=clean_doc(file)
    i=0
    for pages in file_cleaned:
        text_={}
        text_['content']=pages
        text_['id']=i
        if is_title(pages[0]):
            text_['header']=pages[0]
            
        text_structured.append(text_)
        i+=1
                
    return text_section        
            

In [251]:
section=get_structured_document(file)


In [31]:

def word_frequency(text):
    word_frequencies = {}
    for word in nltk.word_tokenize(text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    return word_frequencies

In [32]:
def sentence_score(sentence_list,word_frequencies):
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    return sentence_scores


In [33]:
import heapq
def summary(sentence_scores):
    summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)

    
    return summary

In [ ]:
text_path = "./7100-15/cftc_text_7100-15_order-allied-markets-llc-et-al.txt"
with open(text_path) as f:
    file = f.read()
article_text, formatted_article_text=clean(file)

sentence_list = nltk.sent_tokenize(article_text)
word_frequencies=word_frequency(formatted_article_text)
stopwords = nltk.corpus.stopwords.words('english')

maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
sentence_scores = sentence_score(sentence_list, word_frequencies)
    
summary(sentence_scores)



##  Training textsplit model using word vector

In [192]:
import textsplit

In [274]:
import os
import word2vec
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from textsplit.tools import SimpleSentenceTokenizer
from textsplit.tools import get_penalty, get_segments
from textsplit.algorithm import split_optimal, split_greedy, get_total
%matplotlib inline


In [275]:
corpus_path = './text8'  # be sure your corpus is cleaned from punctuation and lowercased
if not os.path.exists(corpus_path):
    !wget http://mattmahoney.net/dc/text8.zip
    !unzip {corpus_path}

In [276]:
links = './7100-15/cftc_text_7100-15_order-allied-markets-llc-et-al.txt'

In [ ]:
wrdvec_path = 'wrdvecs.bin'
if not os.path.exists(wrdvec_path):
    %time word2vec.word2vec(corpus_path, wrdvec_path, cbow=1, iter_=5, hs=1, threads=8, sample='1e-5', window=15, size=200, binary=1)

In [57]:
model = word2vec.load(wrdvec_path)
wrdvecs = pd.DataFrame(model.vectors, index=model.vocab)
del model
print(wrdvecs.shape)

True

In [58]:
sentence_tokenizer = SimpleSentenceTokenizer()


In [59]:
link = links
segment_len = 30  # segment target length in sentences
book_path = os.path.basename(link)

with open(book_path, 'rt') as f:
    text = f.read()  #.replace('\n', ' ')  # punkt tokenizer handles newlines not so nice

sentenced_text = sentence_tokenizer(text)
vecr = CountVectorizer(vocabulary=wrdvecs.index)

sentence_vectors = vecr.transform(sentenced_text).dot(wrdvecs)

penalty = get_penalty([sentence_vectors], segment_len)
print('penalty %4.2f' % penalty)

optimal_segmentation = split_optimal(sentence_vectors, penalty, seg_limit=250)
segmented_text = get_segments(sentenced_text, optimal_segmentation)

print('%d sentences, %d segments, avg %4.2f sentences per segment' % (
    len(sentenced_text), len(segmented_text), len(sentenced_text) / len(segmented_text)))

with open(book_path + '.seg', 'wt') as f:
    for i, segment_sentences in enumerate(segmented_text):
        segment_str = ' // '.join(segment_sentences)
        gain = optimal_segmentation.gains[i] if i < len(segmented_text) - 1 else 0
        segment_info = ' [%d sentences, %4.3f] ' % (len(segment_sentences), gain) 
        print(segment_str + '\n8<' + '=' * 30 + segment_info + "=" * 30, file=f)

greedy_segmentation = split_greedy(sentence_vectors, max_splits=len(optimal_segmentation.splits))
greedy_segmented_text = get_segments(sentenced_text, greedy_segmentation)
lengths_optimal = [len(segment) for segment in segmented_text for sentence in segment]
lengths_greedy = [len(segment) for segment in greedy_segmented_text for sentence in segment]
df = pd.DataFrame({'greedy':lengths_greedy, 'optimal': lengths_optimal})
df.plot.line(figsize=(18, 3), title='Segment lenghts over text')
df.plot.hist(bins=30, alpha=0.5, figsize=(10, 3), title='Histogram of segment lengths')

totals = [get_total(sentence_vectors, seg.splits, penalty) 
          for seg in [optimal_segmentation, greedy_segmentation]]
print('optimal score %4.2f, greedy score %4.2f' % tuple(totals))
print('ratio of scores %5.4f' % (totals[0] / totals[1]))

## No case/docket

In [61]:
clean_page = clean_doc(file)
clean_page[1][2:4]

[' There is good cause to believe that Defendant Allied Markets LLC ("Allied Markets"), by and through its agents, principals and control persons, Defendants Joshua Gilliland and Chawalit Wongkhiao have engaged in, are engaging in, and may continue to engage in violations of Sections 2(c)(2)(C)(iii)(I)(cc), 4b(a)(2)(A) and (C), 4k(2), 4m(l), and 4o(l) of the CEA, 7 U.S.C. 2(c)(2)(C)(iii)(I)(cc), 6b(a)(2)(A), (C), 6k(2), 6m(l), 60(1) (2012); and CFTC Regulations 3. 12, 4.20(a)-(c), 4.4l(a), 5.2(b)(l) and (3), 5.3(a)(2)(i), and 5.3(a)(2)(ii), 17 C.F.R. 3.12, 4.20(a)-(c), 4.4l(a), 5.2(b)(l), (3), 5.3(a)(2)(i), (ii) (2014).']

In [1]:
def get_case(first_page):
    for line in first_page:
        if len(line) < 40:
            p = re.compile("NO[\.:]\s*.+")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()
            
    for line in first_page:   
        if len(line) < 40:    
            p = re.compile("\d*-?CV-\d+.*")
            result = p.search(line.upper())
            if result:
                return result.group(0).strip()

In [63]:
get_case(clean_page[1])

In [2]:
get_case([' CFTC Docket No. SD 20-01'])

NameError: name 're' is not defined

## Defendant

In [50]:
!pip3 install spacy

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
Defaulting to user installation because normal site-packages is not writeable
distutils: /home/aimsg/.local/include/python3.7m/UNKNOWN
sysconfig: /home/aimsg/.local/include/python3.7
user = True
home = None
root = None
prefix = None


In [51]:
!python3 -m spacy download en_core_web_sm


/home/aimsg/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aimsg/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aimsg/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aimsg/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [65]:
import spacy


# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("\n".join(clean_page[0]))
print(text)
doc = nlp(text)

# Analyze syntax
#print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
#print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    if entity.label_ == "ORG":
        print(entity.text,":       ", entity.label_)

 UNITED STATES DISTRICT COURT
 MIDDLE DISTRICT OF FLORIDA
 Jacksonville Division
 v.
 Case No. 3:15-cv-5-J-34MCR
 U.S. COMMODITY FUTURES
 TRADING COMMISSION,
 Plaintiff,
 ALLIED MARKETS LLC,
 JOSHUA GILLILAND, and
 CHAWALIT WONGKHIAO,
 Defendants.
 ORDER GRANTING PLAINTIFF'S EX PARTE
 MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING
 PRELIMINARY fNJUNCTION HEARING


In [66]:
print("\n".join(clean_page[0]))

 UNITED STATES DISTRICT COURT
 MIDDLE DISTRICT OF FLORIDA
 Jacksonville Division
 v.
 Case No. 3:15-cv-5-J-34MCR
 U.S. COMMODITY FUTURES
 TRADING COMMISSION,
 Plaintiff,
 ALLIED MARKETS LLC,
 JOSHUA GILLILAND, and
 CHAWALIT WONGKHIAO,
 Defendants.
 ORDER GRANTING PLAINTIFF'S EX PARTE
 MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING
 PRELIMINARY fNJUNCTION HEARING


In [44]:
#!pip install transformers
#!pip3 install torch torchvision torchaudio
#!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

In [67]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Question answering pipeline, specifying the checkpoint identifier
nlp = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')

In [68]:
first_page = [text for text in clean_page[0] if len(text) > 5]

In [69]:
first_page

[' UNITED STATES DISTRICT COURT',
 ' MIDDLE DISTRICT OF FLORIDA',
 ' Jacksonville Division',
 ' Case No. 3:15-cv-5-J-34MCR',
 ' U.S. COMMODITY FUTURES',
 ' TRADING COMMISSION,',
 ' Plaintiff,',
 ' ALLIED MARKETS LLC,',
 ' JOSHUA GILLILAND, and',
 ' CHAWALIT WONGKHIAO,',
 ' Defendants.',
 " ORDER GRANTING PLAINTIFF'S EX PARTE",
 ' MOTION FOR STATUTORY RESTRAINING ORDER AND SCHEDULING',
 ' PRELIMINARY fNJUNCTION HEARING']

In [73]:
print(nlp(question="Who violeted?", context=formatted_article_text))

/home/aimsg/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.011641628108918667, 'start': 632, 'end': 642, 'answer': 'Defendants'}


In [76]:
print(nlp(question="Who is the defendant?", context=formatted_article_text, topk=2))

/home/aimsg/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


[{'score': 0.5419595241546631, 'start': 14031, 'end': 14047, 'answer': 'Joshua Gilliland'}, {'score': 0.33723416924476624, 'start': 27324, 'end': 27363, 'answer': 'Joshua Gilliland and Chawalit Wongkhiao'}]


# STOP HERE

## Extracted authorities

In [ ]:
UNITED STATES DISTRICT COURT
FOR THE MIDDLE DISTRICT OF FLORIDA 

## Code law violation

In [ ]:
violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C.
§ 6c(a)(5)(C) (2018).

In [ ]:
Section 4c(a)(5) of the Act, 7 U.S.C.
§ 6c(a)(5) (2012).

## Violation period

In [428]:
line = 'The Commodity Futures Trading Commission ("Commission") has reason to believe that from at least July 2012 through March 2017 ("Relevant Period"), Propex Derivatives Pty Ltd. ("Propex") violated Section 4c(a)(5)(C) of the Commodity Exchange Act ("Act"), 7 U.S.C. 6c(a)(5)(C) (2018). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Propex engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.'

In [ ]:
("Relevant Period")

## Transaction amount

In [ ]:
pour complaint tous les montants vers Transaction amounts

## Defendant

In [ ]:
Propex Derivatives Pty Ltd, Respondent.

In [18]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm

## Nature of violations

In [ ]:
II. FINDINGS
The Commission finds the following:
A. SUMMARY
During the Relevant Period, Propex, by and through a Propex trader ("Trader A"),·
engaged in thousands of instances of the disruptive trading practice known as "spoofing"
(bidding or offering with the intent to cancel the bid or offer before execution) in the E-mini
S&P 500 futures contracts traded on the Chicago Mercantile Exchange ("CME"), a futures
exchange and designated contract market which is owned and operated by CME Group Inc. This
conduct violated Section 4c(a)(5)(C) of the Act, 7 U.S.C. § 6c(a)(5)(C) (2018). 

In [66]:
cftc_full_list = pd.read_excel("cftc_full_list.xlsx")

In [68]:
def gen_line_rep(rep):
    original_path = f"./cftc/original/{rep}"
    text_path = f"./cftc/text/{rep}"
    meta_path = f"./cftc/meta-data/{rep}"
    lines = []
    for file in glob.glob(f"{meta_path}/*.json"):
        filename = os.path.basename(file)

        with open(file) as json_file: 
            data = json.load(json_file)     

        data['filename'] = filename.replace(".json","")
        data['folder'] = rep
        with open(f"{text_path}/{filename.replace('.json','.txt')}") as f:
            data['doc_text'] = f.read()  


        lines.append(data)
    return lines

In [69]:
meta_data = []
for index, row in cftc_full_list[~cftc_full_list.scraped_folder.isnull()].iterrows():
    try:
        meta_data.extend(gen_line_rep(row['scraped_folder']))
    except:
        print("error on",  row['scraped_folder'])
len(meta_data)

error on 7882-19
error on 8262-20
error on 8239-20


756

In [70]:
df_meta_data = pd.DataFrame(meta_data)

In [71]:
df_meta_data['doc_clean'] = df_meta_data.doc_text.apply(clean_doc)

In [72]:
df_meta_data['first_page'] = df_meta_data['doc_clean'].str[0]

In [73]:
df_meta_data = df_meta_data[~df_meta_data['first_page'].isnull()]

## Get reference

In [27]:
df_meta_data['reference'] = df_meta_data.first_page.apply(get_case)

## Get defendant

In [178]:
first_page = [text for text in df_meta_data.first_page.values[3] if len(text) > 100]

In [179]:
first_page

[' The Commodity Futures Trading Commission ("Commission") has reason to believe that from in or about August 2012 to at least January 2015 (the "Relevant Period"), Respondents Sharpe Signa, LLC ("Sharpe") and Garen Ovsepyan ("Ovsepyan") violated Sections 2(c)(2)(C)(iii)(I)(bb), 4b(a)(2)(A) and (C), 4o(l)(A) and (B), and 6(c)(2) of the Act, 7 U.S.C.',
 ' SSSS 2(c)(2)(C)(iii)(I)(bb), 6b(a)(2)(A) and (C), 6o(l)(A) and (B), and 9(2) (2012), and Commission Regulation ("Regulation") 5.3(a)(3)(i), 17 C.F.R. SS 5.3(a)(3)(i) (2014), and Respondent Haeres Capital, LLC ("Haeres") and Ovsepyan violated Section 6(c )(2) of the Act, 7 U.S.C. SS 9(2) (2012). Therefore, the Commission deems it appropriate and in the public interest that public administrative proceedings be, and hereby are, instituted to determine whether Respondent(s) engaged in the violations set forth herein and to determine whether any order should be issued imposing remedial sanctions.',
 ' In anticipation of the institution of a

In [180]:
print(nlp(question="Who violeted?", context=".".join(first_page)))

{'score': 0.1322660595178604, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}


In [181]:
print(nlp(question="Who is the defendant?", context=".".join(first_page), topk=3))

[{'score': 0.009618670679628849, 'start': 208, 'end': 222, 'answer': 'Garen Ovsepyan'}, {'score': 0.0038621597923338413, 'start': 175, 'end': 187, 'answer': 'Sharpe Signa'}, {'score': 0.0027537851128727198, 'start': 175, 'end': 203, 'answer': 'Sharpe Signa, LLC ("Sharpe")'}]


In [141]:
#df_meta_data[['reference', 'folder', 'filename']][df_meta_data.reference.isnull()]

## Type

In [218]:
df_meta_data['is_order'] = df_meta_data.filename.str.contains("order")

In [219]:
df_meta_data['is_complaint'] = df_meta_data.filename.str.contains("complaint")

In [220]:
df_meta_data['type'] = np.where(df_meta_data['is_order'], "Order CFTC", 
         np.where(df_meta_data['is_complaint'], 
                  "Complaint CFTC", None))